# Demo of Device Emulator for IQM Garnet

## Install the necessary branch

In [1]:
# %pip install types-pkg-resources==0.1.3 pytket
# %pip uninstall -y amazon-braket-sdk
# %pip install git+https://github.com/Altanali/amazon-braket-sdk-python.git@f486203d8d2b00483d49649b7b78073e9ee1c238
# %pip uninstall -y amazon-braket-default-simulator
# %pip install git+https://github.com/amazon-braket/amazon-braket-default-simulator-python.git@c333ad708dd18142001fdc7675f538c7e4cd94c4

In [2]:
%pip freeze | grep "pytket\|amazon"

amazon-braket-algorithm-library==1.4.13
amazon-braket-default-simulator @ git+https://github.com/amazon-braket/amazon-braket-default-simulator-python.git@c333ad708dd18142001fdc7675f538c7e4cd94c4
amazon-braket-pennylane-plugin==1.28.0
amazon-braket-schemas==1.22.0
amazon-braket-sdk @ git+https://github.com/Altanali/amazon-braket-sdk-python.git@f486203d8d2b00483d49649b7b78073e9ee1c238
pytket==1.22.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
# It should output
# amazon-braket-algorithm-library==1.4.13
# amazon-braket-default-simulator @ git+https://github.com/amazon-braket/amazon-braket-default-simulator-python.git@c333ad708dd18142001fdc7675f538c7e4cd94c4
# amazon-braket-pennylane-plugin==1.28.0
# amazon-braket-schemas==1.22.0
# amazon-braket-sdk @ git+https://github.com/Altanali/amazon-braket-sdk-python.git@f486203d8d2b00483d49649b7b78073e9ee1c238
# pytket==1.22.0

## Define the quantum circuit we want to run from qiskit

In [4]:
from qiskit import QuantumCircuit

def ghz(n, cnot=True):
    circuit = QuantumCircuit(n)
    circuit.h(0)
    for i in range(1, n):
        if cnot:
            circuit.cx(0, i)
        else:
            circuit.cz(0, i)
    
    return circuit    

qc = ghz(2)
qc.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

## Convert it to braket circuit class

In [5]:
from qiskit_braket_provider import to_braket

qc2 = to_braket(qc)
print(qc2)

T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │


## Define the IQM device 
Garnet is the only device that has properly working device emulator now


In [6]:

from braket.aws import AwsDevice
from braket.devices import Devices

garnet = AwsDevice(Devices.IQM.Garnet)

## Run emulation

In [7]:
result_emu = garnet.emulate(qc2, shots=1000).result()
print(result_emu.measurement_counts)

print(result_emu.measured_qubits)

Counter({'11': 502, '00': 490, '01': 4, '10': 4})
[1, 4]


The presence of 01, 10 indicates the presence of noise. Also note the label for the measured qubits, see below

## Inspect the emulated circuit

### A few things to note
1. The circuit has been nativized to use the native gates of Garnet, namely PRX and CZ gates
2. The circuit has been compiled based on the device geometry
3. Each gate has been appended with certain noise channels

In [8]:
qc3 = garnet.run_passes(qc2)
print(qc3)


T  : │                                0                                 │          1           │                                2                                │
      ┌─────────────────┐ ┌────────────┐ ┌────────────┐ ┌─────────────┐  ┌───┐ ┌──────────────┐ ┌─────────────────┐ ┌────────────┐ ┌────────────┐ ┌─────────────┐ 
q1 : ─┤ PRx(1.57, 4.71) ├─┤ AD(0.0011) ├─┤ PD(0.0022) ├─┤ DEPO(0.001) ├──┤ Z ├─┤ DEPO(0.0073) ├─┤ PRx(1.57, 1.57) ├─┤ AD(0.0011) ├─┤ PD(0.0022) ├─┤ DEPO(0.001) ├─
      └─────────────────┘ └────────────┘ └────────────┘ └─────────────┘  └─┬─┘ └──────┬───────┘ └─────────────────┘ └────────────┘ └────────────┘ └─────────────┘ 
      ┌─────────────────┐ ┌────────────┐ ┌────────────┐ ┌──────────────┐   │   ┌──────┴───────┐                                                                   
q4 : ─┤ PRx(1.57, 4.71) ├─┤ AD(0.0017) ├─┤ PD(0.0024) ├─┤ DEPO(0.0013) ├───●───┤ DEPO(0.0073) ├───────────────────────────────────────────────────────────────────
      └───────────────

# Another example of ghz with CZ gate

In [9]:
qc = ghz(4, cnot=False)
qc.draw()

┌───┐         
q_0: ┤ H ├─■──■──■─
     └───┘ │  │  │ 
q_1: ──────■──┼──┼─
              │  │ 
q_2: ─────────■──┼─
                 │ 
q_3: ────────────■─

In [10]:
qc2 = to_braket(qc, basis_gates=["cz", "h"])
print(qc2)

T  : │  0  │  1  │  2  │  3  │
      ┌───┐                   
q0 : ─┤ H ├───●─────●─────●───
      └───┘   │     │     │   
            ┌─┴─┐   │     │   
q1 : ───────┤ Z ├───┼─────┼───
            └───┘   │     │   
                  ┌─┴─┐   │   
q2 : ─────────────┤ Z ├───┼───
                  └───┘   │   
                        ┌─┴─┐ 
q3 : ───────────────────┤ Z ├─
                        └───┘ 
T  : │  0  │  1  │  2  │  3  │


In [11]:
qc3 = garnet.run_passes(qc2)
print(qc3)


T  : │                                0                                 │          1           │          2           │          3          │
                                                                         ┌───┐ ┌──────────────┐                                              
q1 : ────────────────────────────────────────────────────────────────────┤ Z ├─┤ DEPO(0.0073) ├──────────────────────────────────────────────
                                                                         └─┬─┘ └──────┬───────┘                                              
      ┌─────────────────┐ ┌────────────┐ ┌────────────┐ ┌──────────────┐   │   ┌──────┴───────┐       ┌──────────────┐       ┌─────────────┐ 
q4 : ─┤ PRx(1.57, 4.71) ├─┤ AD(0.0017) ├─┤ PD(0.0024) ├─┤ DEPO(0.0013) ├───●───┤ DEPO(0.0073) ├───●───┤ DEPO(0.0083) ├───●───┤ DEPO(0.024) ├─
      └─────────────────┘ └────────────┘ └────────────┘ └──────────────┘       └──────────────┘   │   └──────┬───────┘   │   └──────┬──────┘ 
      

In [12]:
result_emu = garnet.emulate(qc2, shots=1000).result()
print(result_emu.measurement_counts)

print(result_emu.measured_qubits)

Counter({'0000': 501, '0100': 475, '0101': 9, '0001': 8, '0110': 3, '0010': 3, '1100': 1})
[1, 4, 5, 9]
